<a href="https://colab.research.google.com/github/aldder/MachineLearning/blob/master/Keras%20Tuner/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 61kB 3.4MB/s 


In [6]:
import tensorflow as tf
import kerastuner as kt
tf.__version__, kt.__version__

('2.3.0', '1.0.1')

In [7]:
from sklearn.datasets import load_iris
(x, y) = load_iris(return_X_y=True)

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [9]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [25]:
x_train.shape

(105, 4)

In [28]:
def build_model(hp):
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    for i in range(hp.Int('num_layers', 1, 15)):
        if i==0:
            model.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=5, max_value=150, step=1), activation='relu', input_shape=(4,)))
        else:
            model.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=5, max_value=150, step=1), activation='relu'))
    if hp.Choice('use_dropout', values = [0, 1]):
        model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=0.00001, max_value=0.1, step=0.01)),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [43]:
!rm -r test

In [44]:
import IPython
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

tuner = kt.tuners.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    project_name='test')

epochs = 30
validation_data = (x_test, y_test)
batch_size = 32
verbose = 0
callbacks = [ClearTrainingOutput(), es]

tuner.search(x_train, y_train, 
             epochs=epochs, 
             validation_data=validation_data, 
             batch_size=batch_size, 
             verbose=verbose, 
             callbacks=callbacks)

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [45]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

In [46]:
best_hps.values

{'dropout_rate': 0.5,
 'learning_rate': 0.09000999999999999,
 'num_layers': 1,
 'units_0': 150,
 'units_1': 5,
 'units_10': 150,
 'units_11': 5,
 'units_12': 5,
 'units_13': 5,
 'units_14': 150,
 'units_2': 150,
 'units_3': 150,
 'units_4': 5,
 'units_5': 5,
 'units_6': 150,
 'units_7': 5,
 'units_8': 150,
 'units_9': 150,
 'use_dropout': 0}

In [48]:
final_model = tuner.get_best_models()[0]

final_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 150)               750       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 453       
Total params: 1,203
Trainable params: 1,203
Non-trainable params: 0
_________________________________________________________________


In [49]:
final_model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 0.0316 - accuracy: 1.0000


[0.03159720078110695, 1.0]